In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                170       
Total params: 16,714
Trainable params: 16,714
Non-trainable params: 0
_________________________________________________________________


In [8]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

h 349/550
2/2 [==============================] - 0s 4ms/step - loss: 1.1065 - accuracy: 0.6673
Epoch 350/550
2/2 [==============================] - 0s 3ms/step - loss: 1.0948 - accuracy: 0.6673
Epoch 351/550
2/2 [==============================] - 0s 3ms/step - loss: 1.1079 - accuracy: 0.6568
Epoch 352/550
2/2 [==============================] - 0s 5ms/step - loss: 1.1080 - accuracy: 0.6568
Epoch 353/550
2/2 [==============================] - 0s 2ms/step - loss: 1.1129 - accuracy: 0.6464
Epoch 354/550
2/2 [==============================] - 0s 3ms/step - loss: 1.0872 - accuracy: 0.6568
Epoch 355/550
2/2 [==============================] - 0s 3ms/step - loss: 1.1038 - accuracy: 0.6464
Epoch 356/550
2/2 [==============================] - 0s 4ms/step - loss: 1.0807 - accuracy: 0.6568
Epoch 357/550
2/2 [==============================] - 0s 3ms/step - loss: 1.0667 - accuracy: 0.6568
Epoch 358/550
2/2 [==============================] - 0s 3ms/step - loss: 1.0817 - accuracy: 0.6568
Epoch 359/550


In [9]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets
